In [1]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ["CONDA_PREFIX"] = "/usr/local"
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
#for so in ['cudf', 'rmm', 'nccl']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
# fix for BlazingSQL import issue
# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by /usr/local/lib/python3.7/site-packages/../../libblazingsql-engine.so)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 7.39 MiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
	os.environ['CONDA_PREFIX'] = '/usr/local'
	for so in ['cudf', 'rmm', 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import cudf
import numpy as np

In [43]:
# fisrt 53 columns
# cols with strings
str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5',
            'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

cols = ['TransactionID', 'TransactionDT', 'TransactionAmt',
        'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
        'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
        'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
        'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
        'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
        'M5', 'M6', 'M7', 'M8', 'M9']

v = ['V1', 'V3', 'V4', 'V6', 'V9', 'V11']
v += ['V13', 'V14', 'V17', 'V20', 'V23', 'V26', 'V27', 'V30']
v += ['V36', 'V37', 'V40', 'V41', 'V44', 'V47', 'V48']
v += ['V165', 'V160']
v += ['V332', 'V325', 'V326', 'V335', 'V338']
v += ['V138', 'V139', 'V142', 'V147', 'V155', 'V162']

v += ['V220', 'V221', 'V234', 'V238', 'V250', 'V271'] 
v += ['V281', 'V283', 'V289', 'V296', 'V301', 'V314']  

v += ['V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89', 'V91']  # b1
v += ['V203', 'V207', 'V173', 'V181', 'V192', 'V199', 'V205', 'V216', 'V215']  # b1
v += ['V169', 'V171', 'V174', 'V175', 'V209', 'V185', 'V188', 'V195', 'V198', 'V210']  # b2
v += ['V54', 'V55', 'V56', 'V59', 'V62', 'V65', 'V67', 'V68', 'V70']  # b2
v += ['V307', 'V284', 'V285', 'V286', 'V291', 'V297', 'V303', 'V305', 'V312', 'V310', 'V320'] # b2
v += ['V265', 'V225', 'V264', 'V226', 'V230', 'V240', 'V258', 'V252', 'V262', 'V268', 'V277']  # b3
v += ['V127', 'V98', 'V99', 'V104', 'V107', 'V108', 'V109', 'V111', 'V115', 'V117', 'V118', 'V120', 'V121', 'V123', 'V124', 'V131', 'V130', 'V136']  # b3

cols += [x for x in v]

In [44]:
import os
PROJECT_DIR = os.path.abspath('/content/drive/MyDrive')
DATA_DIR = os.path.join(PROJECT_DIR, 'ieee-fraud-detection')

path_train_transaction = os.path.join(DATA_DIR, 'train_transaction.csv')
path_train_identity = os.path.join(DATA_DIR, 'train_identity.csv')
path_test_transaction = os.path.join(DATA_DIR, 'test_transaction.csv')
path_test_identity = os.path.join(DATA_DIR, 'test_identity.csv')

train_trans = cudf.read_csv(path_train_transaction, index_col="TransactionID", usecols = cols + ["isFraud"])
train_id = cudf.read_csv(path_train_identity, index_col="TransactionID")
X_train = train_trans.merge(train_id, how="left", left_index=True, right_index=True)

test_trans = cudf.read_csv(path_test_transaction, index_col="TransactionID", usecols = cols)
test_id = cudf.read_csv(path_test_identity, index_col="TransactionID")
X_test = test_trans.merge(test_id, how="left", left_index=True, right_index=True)

In [45]:
y_train = X_train["isFraud"]
del train_id, test_id, X_train["isFraud"]

In [46]:
print("X_train shape:{}, X_test shape:{}".format(X_train.shape, X_test.shape))

X_train shape:(590540, 215), X_test shape:(506691, 215)


In [47]:
# transform D feature "time delta" as "time point"
for i in range(1, 16):
    if i in [1, 2, 3, 5, 9]:
        continue
    X_train["D" + str(i)] = X_train["D" + str(i)] - X_train["TransactionDT"] / np.float32(60 * 60 * 24)
    X_test["D" + str(i)] = X_test["D" + str(i)] - X_test["TransactionDT"] / np.float32(60 * 60 * 24)

In [48]:
# change test set id-01 to id_01
train_id_col1 = [f'id_0{i}' for i in range(1, 10)] 
train_id_col2 = [f'id_{i}' for i in range(10, 39)] 
train_id_col = train_id_col1+train_id_col2

test_id_col1 = [f'id-0{i}' for i in range(1, 10)] 
test_id_col2 = [f'id-{i}' for i in range(10, 39)] 
test_id_col = test_id_col1+test_id_col2

new_names = train_id_col
old_names = test_id_col
rename_dict = {old_names[i]: new_names[i] for i in range(len(new_names))}

X_test.rename(columns = rename_dict, inplace=True)

In [49]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    print("Original memory usage: {:.2f} Mb".format(start_mem))

    for col in df.columns:
        try:
            col_type = str(df[col].iloc[0].dtype)
        except:
            col_type = str(df[col].dtype)
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Memory usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [50]:
X_train, X_test = reduce_mem_usage(X_train), reduce_mem_usage(X_test)

Original memory usage: 943.94 Mb
Memory usage decreased to 528.31 Mb (44.0% reduction)
Original memory usage: 811.36 Mb
Memory usage decreased to 454.74 Mb (44.0% reduction)


In [51]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in X_train.columns:
    try:
        col_type = str(X_train[col].iloc[0].dtype)
    except:
        col_type = str(X_train[col].dtype)
    
    if col_type in numerics:
        mn = np.min((X_train[col].min(), X_test[col].min()))
        X_train[col] -= np.float32(mn)
        X_test[col] -= np.float32(mn)
        X_train[col].fillna(-1,inplace=True)
        X_test[col].fillna(-1,inplace=True)

In [52]:
print("Memory further decreased to: {:.2f} mb".format(X_train.memory_usage().sum()/1024**2))
print("Memory further decreased to: {:.2f} mb".format(X_test.memory_usage().sum()/1024**2))

Memory further decreased to: 519.92 mb
Memory further decreased to: 447.79 mb


In [53]:
del train_id_col1, train_id_col2, train_id_col, test_id_col1, test_id_col2, test_id_col, new_names, old_names, rename_dict

In [54]:
# frequency encode
def encode_FE(df1, df2, cols):
    print("frequency encoding")
    for col in cols:
        df = cudf.concat([df1[col], df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_pandas().to_dict()
        nm = col + "_FE"
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype("float32")
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype("float32")
        print(col)

In [55]:
X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32')
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')

In [56]:
fea_cols = ['addr1', 'card1', 'card2', 'card3', 'P_emaildomain']

import time
start = time.time()
encode_FE(X_train, X_test, cols=fea_cols)
elapsed = (time.time() - start)
print("Time used:",elapsed)


X_train[['addr1_FE', 'card1_FE', 'card2_FE', 'card3_FE', 'P_emaildomain_FE']].head()

frequency encoding
addr1
card1
card2
card3
P_emaildomain
Time used: 0.3217942714691162


,addr1_FE,card1_FE,card2_FE,card3_FE,P_emaildomain_FE
TransactionID,,,,,
2989413,0.007863,0.000025,0.016029,0.872054,0.076117493
2989407,0.024794,0.003064,0.075223,0.872054,<NA>
2989400,0.039221,0.002445,0.024812,0.872054,0.076117493
2989416,0.032745,0.006958,0.010287,0.872054,0.003681515
2989408,0.025437,0.000034,0.073279,0.872054,0.006126932


In [57]:
import gc
# label encode
def encode_LE(col, train, test, verbose=True):
    print("label encoding")
    df_comb = cudf.concat([train[col], test[col]], axis=0)
    df_comb, _ = cudf.factorize(df_comb)
    nm = col
    if df_comb.max() > 32000:
        train[nm] = df_comb[0: len(train)].astype("int32")
        test[nm] = df_comb[len(train):].astype("int32")
    else:
        train[nm] = df_comb[0: len(train)].astype("int16")
        test[nm] = df_comb[len(train):].astype("int16")
    del df_comb
    gc.collect()
    if verbose:
        print(col)

# COMBINE FEATURES
def encode_CB(col1, col2, df1, df2):
    print("combine features")
    nm = col1 + '_' + col2
    df1[nm] = df1[col1].astype(str) + '_' + df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str) + '_' + df2[col2].astype(str)
    encode_LE(nm, df1, df2, verbose=False)
    print(nm, ', ', end='')

In [58]:
start = time.time()
encode_CB('card1', 'addr1', X_train, X_test)
elapsed = (time.time() - start)
print("Time used:",elapsed)


X_train[['card1', 'addr1', 'card1_addr1']].head()

combine features
label encoding
card1_addr1 , Time used: 0.197523832321167


,card1,addr1,card1_addr1
TransactionID,,,
2989413,16541.0,77.0,23025
2989407,2821.0,84.0,29663
2989400,893.0,215.0,51379
2989416,17132.0,172.0,24883
2989408,8839.0,91.0,51083


In [59]:
start = time.time()
encode_CB('card1_addr1', 'P_emaildomain', X_train, X_test)
elapsed = (time.time() - start)
print("Time used:",elapsed)


X_train[['card1_addr1', 'P_emaildomain', 'card1_addr1_P_emaildomain']].head()

combine features
label encoding
card1_addr1_P_emaildomain , Time used: 0.17722845077514648


,card1_addr1,P_emaildomain,card1_addr1_P_emaildomain
TransactionID,,,
2989413,23025,anonymous.com,30176
2989407,29663,<NA>,-1
2989400,51379,anonymous.com,95281
2989416,24883,bellsouth.net,33998
2989408,51083,live.com,94696


In [60]:
fea_cols = ['card1_addr1', 'card1_addr1_P_emaildomain']

start = time.time()
encode_FE(X_train, X_test, fea_cols)
elapsed = (time.time() - start)
print("Time used:",elapsed)

X_train[['card1_addr1_FE', 'card1_addr1_P_emaildomain_FE']].head()

frequency encoding
card1_addr1
card1_addr1_P_emaildomain
Time used: 0.22173643112182617


,card1_addr1_FE,card1_addr1_P_emaildomain_FE
TransactionID,,
2989413,0.000022,0.000004
2989407,0.000070,0.149146
2989400,0.000156,0.000009
2989416,0.001138,0.000029
2989408,0.000033,0.000002


In [61]:
def encode_AG(main_columns, uids, df_train, df_test, aggregations=["mean"], fillna=True, usena=False):
    for main_column in main_columns:
        print(f"\nmain_column: {main_column}")
        for col in uids:
            print(f"col: {col}")
            for agg_type in aggregations:
                print(f"agg_type: {agg_type}")
                new_column = main_column + "_" + col + "_" + agg_type
                print(f"new_column: {new_column}")
                temp_df = cudf.concat([df_train[[col, main_column]], df_test[[col, main_column]]])
                if usena:
                    temp_df.loc[temp_df[main_column] == -1, main_column] = np.nan


                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                    columns={agg_type: new_column})
                temp_df.index = list(temp_df[col].to_pandas())
                temp_df = temp_df[new_column].to_pandas().to_dict()
                df_train[new_column] = df_train[col].map(temp_df).astype("float32")
                df_test[new_column] = df_test[col].map(temp_df).astype("float32")
                if fillna:
                    df_train[new_column].fillna(-1, inplace=True)
                    df_test[new_column].fillna(-1, inplace=True)

In [62]:
new_cols = [
    'TransactionAmt_card1_mean',
    'TransactionAmt_card1_std',
    'TransactionAmt_card1_addr1_mean',
    'TransactionAmt_card1_addr1_std',
    'TransactionAmt_card1_addr1_P_emaildomain_mean',
    'TransactionAmt_card1_addr1_P_emaildomain_std',
    'D9_card1_mean',
    'D9_card1_std',
    'D9_card1_addr1_mean',
    'D9_card1_addr1_std',
    'D9_card1_addr1_P_emaildomain_mean',
    'D9_card1_addr1_P_emaildomain_std',
    'D11_card1_mean',
    'D11_card1_std',
    'D11_card1_addr1_mean',
    'D11_card1_addr1_std',
    'D11_card1_addr1_P_emaildomain_mean',
    'D11_card1_addr1_P_emaildomain_std'
]

In [63]:
start = time.time()
encode_AG(
    ['TransactionAmt', 'D9', 'D11'], 
    ['card1', 'card1_addr1', 'card1_addr1_P_emaildomain'], 
    X_train, X_test,
    ['mean', 'std'],
    usena=False
)
elapsed = (time.time() - start)
print("Time used:",elapsed)


X_train[new_cols].head()


main_column: TransactionAmt
col: card1
agg_type: mean
new_column: TransactionAmt_card1_mean
agg_type: std
new_column: TransactionAmt_card1_std
col: card1_addr1
agg_type: mean
new_column: TransactionAmt_card1_addr1_mean
agg_type: std
new_column: TransactionAmt_card1_addr1_std
col: card1_addr1_P_emaildomain
agg_type: mean
new_column: TransactionAmt_card1_addr1_P_emaildomain_mean
agg_type: std
new_column: TransactionAmt_card1_addr1_P_emaildomain_std

main_column: D9
col: card1
agg_type: mean
new_column: D9_card1_mean
agg_type: std
new_column: D9_card1_std
col: card1_addr1
agg_type: mean
new_column: D9_card1_addr1_mean
agg_type: std
new_column: D9_card1_addr1_std
col: card1_addr1_P_emaildomain
agg_type: mean
new_column: D9_card1_addr1_P_emaildomain_mean
agg_type: std
new_column: D9_card1_addr1_P_emaildomain_std

main_column: D11
col: card1
agg_type: mean
new_column: D11_card1_mean
agg_type: std
new_column: D11_card1_std
col: card1_addr1
agg_type: mean
new_column: D11_card1_addr1_mean
agg_

,TransactionAmt_card1_mean,TransactionAmt_card1_std,TransactionAmt_card1_addr1_mean,TransactionAmt_card1_addr1_std,TransactionAmt_card1_addr1_P_emaildomain_mean,TransactionAmt_card1_addr1_P_emaildomain_std,D9_card1_mean,D9_card1_std,D9_card1_addr1_mean,D9_card1_addr1_std,D9_card1_addr1_P_emaildomain_mean,D9_card1_addr1_P_emaildomain_std,D11_card1_mean,D11_card1_std,D11_card1_addr1_mean,D11_card1_addr1_std,D11_card1_addr1_P_emaildomain_mean,D11_card1_addr1_P_emaildomain_std
TransactionID,,,,,,,,,,,,,,,,,,
2989413,124.464935,66.182755,126.531555,70.081184,99.982002,0.000000,-0.876543,0.444800,-0.861111,0.470550,-0.166667,0.962250,225.069138,210.206573,224.931900,215.723770,-1.000000,0.000000
2989407,228.850037,361.889343,191.661346,297.586365,118.041283,192.946793,-0.697737,0.634562,-0.567100,0.740934,-0.825071,0.504442,189.949524,247.391190,149.771957,244.241592,265.942017,269.591278
2989400,269.247986,393.990845,270.911194,384.716705,523.959045,682.289795,-0.842108,0.491226,-0.819932,0.517438,-1.000000,0.000000,223.007462,237.794708,282.277863,301.959015,326.402771,325.760376
2989416,120.945419,196.463486,116.219604,155.025528,111.983543,67.203819,-0.920148,0.351410,-0.962403,0.244122,-1.000000,0.000000,288.241089,270.982452,281.910431,251.790894,393.544769,303.444946
2989408,68.288498,67.264969,69.102570,68.034027,81.982002,7.071068,-0.823198,0.415671,-0.818287,0.420477,-1.000000,0.000000,164.347809,189.385956,163.715469,192.032791,151.664856,215.900711


In [64]:
for col in str_type:
    encode_LE(col, X_train, X_test)

label encoding
ProductCD
label encoding
card4
label encoding
card6
label encoding
P_emaildomain
label encoding
R_emaildomain
label encoding
M1
label encoding
M2
label encoding
M3
label encoding
M4
label encoding
M5
label encoding
M6
label encoding
M7
label encoding
M8
label encoding
M9
label encoding
id_12
label encoding
id_15
label encoding
id_16
label encoding
id_23
label encoding
id_27
label encoding
id_28
label encoding
id_29
label encoding
id_30
label encoding
id_31
label encoding
id_33
label encoding
id_34
label encoding
id_35
label encoding
id_36
label encoding
id_37
label encoding
id_38
label encoding
DeviceType
label encoding
DeviceInfo


In [65]:
cols = list(X_train.columns)
cols.remove('TransactionDT')
for c in ['D6', 'D7', 'D8', 'D9', 'D12', 'D13', 'D14']:
    cols.remove(c)

# FAILED TIME CONSISTENCY TEST
for c in ['C3', 'M5', 'id_08', 'id_33']:
    cols.remove(c)
for c in ['card4', 'id_07', 'id_14', 'id_21', 'id_30', 'id_32', 'id_34']:
    cols.remove(c)
for c in ['id_' + str(x) for x in range(22, 28)]:
    cols.remove(c)

print('NOW USING THE FOLLOWING', len(cols), 'FEATURES.')
# CHRIS - TRAIN 75% PREDICT 25%
idxT = X_train.index[:3 * len(X_train) // 4]
idxV = X_train.index[3 * len(X_train) // 4:]
print(X_train.info())
# X_train = X_train.convert_objects(convert_numeric=True)
# X_test = X_test.convert_objects(convert_numeric=True)

for col in str_type:
    print(col)
    X_train[col] = X_train[col].astype(int)
    X_test[col] = X_test[col].astype(int)
print("after transform:")
print(X_train.info())

# fillna
for col in cols:
    X_train[col].fillna(-1, inplace=True)
    X_test[col].fillna(-1, inplace=True)

X_train.fillna(-1, inplace = True)
X_test.fillna(-1, inplace = True)

NOW USING THE FOLLOWING 218 FEATURES.
<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 590540 entries, 2989413 to 3575847
Columns: 243 entries, TransactionDT to D11_card1_addr1_P_emaildomain_std
dtypes: float32(209), float64(1), int16(31), int32(2)
memory usage: 519.3 MB
None
ProductCD
card4
card6
P_emaildomain
R_emaildomain
M1
M2
M3
M4
M5
M6
M7
M8
M9
id_12
id_15
id_16
id_23
id_27
id_28
id_29
id_30
id_31
id_33
id_34
id_35
id_36
id_37
id_38
DeviceType
DeviceInfo
after transform:
<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 590540 entries, 2989413 to 3575847
Columns: 243 entries, TransactionDT to D11_card1_addr1_P_emaildomain_std
dtypes: float32(209), float64(1), int32(2), int64(31)
memory usage: 624.1 MB
None


In [ ]:
PROJECT_DIR = os.path.abspath('/content/drive/MyDrive')
SAVE_DIR = os.path.join(PROJECT_DIR, 'ieee-fraud-detection/train_val')

X_train.loc[idxT, cols].to_csv(os.path.join(SAVE_DIR, "X_train.csv"))
y_train.loc[idxT].reset_index().to_csv(os.path.join(SAVE_DIR, "y_train.csv"), index = False)
X_train.loc[idxV, cols].to_csv(os.path.join(SAVE_DIR, "X_val.csv"))
y_train.loc[idxV].reset_index().to_csv(os.path.join(SAVE_DIR, "y_val.csv"), index = False)

In [31]:
PROJECT_DIR = os.path.abspath('/content/drive/MyDrive')
SAVE_DIR = os.path.join(PROJECT_DIR, 'ieee-fraud-detection/train_val')

X_test.to_csv(os.path.join(SAVE_DIR, "X_test.csv"))

## UID encoding

In [66]:
# GROUP AGGREGATION NUNIQUE
def encode_AG2(main_columns, uids, df_train, df_test):
    for main_column in main_columns:  
        for col in uids:
            comb = cudf.concat([df_train[[col]+[main_column]],df_test[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_pandas().to_dict()
            df_train[col+'_'+main_column+'_ct'] = df_train[col].map(mp).astype('float32')
            df_test[col+'_'+main_column+'_ct'] = df_test[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

In [67]:
X_train['day'] = X_train.TransactionDT / (24*60*60)
X_train['uid'] = X_train.card1_addr1.astype(str)+'_'+np.floor(X_train.day-X_train.D1).astype(str)

X_test['day'] = X_test.TransactionDT / (24*60*60)
X_test['uid'] = X_test.card1_addr1.astype(str)+'_'+np.floor(X_test.day-X_test.D1).astype(str)

In [68]:
%%time
# FREQUENCY ENCODE UID
encode_FE(X_train,X_test,['uid'])
# AGGREGATE 
encode_AG(['TransactionAmt','D4','D9','D10','D15'],['uid'], X_train, X_test,['mean','std'], fillna=True, usena=True)

frequency encoding
uid

main_column: TransactionAmt
col: uid
agg_type: mean
new_column: TransactionAmt_uid_mean
agg_type: std
new_column: TransactionAmt_uid_std

main_column: D4
col: uid
agg_type: mean
new_column: D4_uid_mean
agg_type: std
new_column: D4_uid_std

main_column: D9
col: uid
agg_type: mean
new_column: D9_uid_mean
agg_type: std
new_column: D9_uid_std

main_column: D10
col: uid
agg_type: mean
new_column: D10_uid_mean
agg_type: std
new_column: D10_uid_std

main_column: D15
col: uid
agg_type: mean
new_column: D15_uid_mean
agg_type: std
new_column: D15_uid_std
CPU times: user 5.82 s, sys: 873 ms, total: 6.7 s
Wall time: 6.59 s


In [69]:
# AGGREGATE
encode_AG(['C'+str(x) for x in range(1,15) if x!=3],['uid'], X_train, X_test, ['mean'], fillna=True, usena=True)


main_column: C1
col: uid
agg_type: mean
new_column: C1_uid_mean

main_column: C2
col: uid
agg_type: mean
new_column: C2_uid_mean

main_column: C4
col: uid
agg_type: mean
new_column: C4_uid_mean

main_column: C5
col: uid
agg_type: mean
new_column: C5_uid_mean

main_column: C6
col: uid
agg_type: mean
new_column: C6_uid_mean

main_column: C7
col: uid
agg_type: mean
new_column: C7_uid_mean

main_column: C8
col: uid
agg_type: mean
new_column: C8_uid_mean

main_column: C9
col: uid
agg_type: mean
new_column: C9_uid_mean

main_column: C10
col: uid
agg_type: mean
new_column: C10_uid_mean

main_column: C11
col: uid
agg_type: mean
new_column: C11_uid_mean

main_column: C12
col: uid
agg_type: mean
new_column: C12_uid_mean

main_column: C13
col: uid
agg_type: mean
new_column: C13_uid_mean

main_column: C14
col: uid
agg_type: mean
new_column: C14_uid_mean


In [70]:
# AGGREGATE
encode_AG(['M'+str(x) for x in range(1,10)],['uid'], X_train, X_test, ['mean'], fillna=True,usena=True)


main_column: M1
col: uid
agg_type: mean
new_column: M1_uid_mean

main_column: M2
col: uid
agg_type: mean
new_column: M2_uid_mean

main_column: M3
col: uid
agg_type: mean
new_column: M3_uid_mean

main_column: M4
col: uid
agg_type: mean
new_column: M4_uid_mean

main_column: M5
col: uid
agg_type: mean
new_column: M5_uid_mean

main_column: M6
col: uid
agg_type: mean
new_column: M6_uid_mean

main_column: M7
col: uid
agg_type: mean
new_column: M7_uid_mean

main_column: M8
col: uid
agg_type: mean
new_column: M8_uid_mean

main_column: M9
col: uid
agg_type: mean
new_column: M9_uid_mean


In [71]:
# AGGREGATE
encode_AG2(['P_emaildomain','dist1','id_02','cents'], ['uid'], X_train, X_test)
# AGGREGATE
encode_AG(['C14'],['uid'], X_train, X_test,['std'], fillna=True,usena=True)
# AGGREGATE 
encode_AG2(['C13','V314'], ['uid'], X_train, X_test)
# AGGREATE 
encode_AG2(['V127','V136','V307','V320'], ['uid'], X_train, X_test)
# NEW FEATURE
X_train['outsider15'] = (np.abs(X_train.D1-X_train.D15)>3).astype('int16')
X_test['outsider15'] = (np.abs(X_test.D1-X_test.D15)>3).astype('int16')
print('outsider15')

uid_P_emaildomain_ct, uid_dist1_ct, uid_id_02_ct, uid_cents_ct, 
main_column: C14
col: uid
agg_type: std
new_column: C14_uid_std
uid_C13_ct, uid_V314_ct, uid_V127_ct, uid_V136_ct, uid_V307_ct, uid_V320_ct, outsider15


In [73]:
# CHRIS - TRAIN 75% PREDICT 25%
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

In [74]:
PROJECT_DIR = os.path.abspath('/content/drive/MyDrive')
SAVE_DIR = os.path.join(PROJECT_DIR, 'ieee-fraud-detection/train_val_uid')

X_train.loc[idxT, cols].to_csv(os.path.join(SAVE_DIR, "X_train.csv"))
y_train.loc[idxT].reset_index().to_csv(os.path.join(SAVE_DIR, "y_train.csv"), index = False)
X_train.loc[idxV, cols].to_csv(os.path.join(SAVE_DIR, "X_val.csv"))
y_train.loc[idxV].reset_index().to_csv(os.path.join(SAVE_DIR, "y_val.csv"), index = False)

In [75]:
PROJECT_DIR = os.path.abspath('/content/drive/MyDrive')
SAVE_DIR = os.path.join(PROJECT_DIR, 'ieee-fraud-detection/train_val_uid')

X_test.to_csv(os.path.join(SAVE_DIR, "X_test.csv"))